In [54]:
!pip install obspy wget

In [61]:
import obspy
import numpy as np
import scipy
import pandas as pd
import h5py
from obspy.clients.fdsn.client import Client 
client= Client("IRIS")
from scipy.signal import decimate, butter,filtfilt
import matplotlib.pyplot as plt
import wget
import scipy.signal as signal
from obspy.geodetics.base import gps2dist_azimuth
from obspy.geodetics import locations2degrees
import matplotlib.dates as mdates

# datetime interaction
from datetime import datetime
from datetime import timedelta as dt
from obspy.core.utcdatetime import UTCDateTime

In [68]:
df = pd.read_csv("C:/Users/wolfn/.seisbench/datasets/comcat/comcat_metadata.csv")
df2 = pd.read_csv("C:/Users/wolfn/.seisbench/datasets/comcat/miniPNW_metadata.csv")

In [113]:
df['preferred_source_magnitude_type'].unique()

array(['md', 'ml', 'mh'], dtype=object)

In [63]:
f = h5py.File("C:/Users/wolfn/.seisbench/datasets/comcat/comcat_waveforms.hdf5", "r")
f2 = h5py.File("C:/Users/wolfn/.seisbench/datasets/comcat/miniPNw_waveforms.hdf5", "r")

In [85]:
# Calculate P/S ratios for all events labeled explosion

# df = df[df['source_type'] == 'explosion']
df['source_origin_time'][1]

'2002-10-03T01:56:49.530000Z'

In [77]:
def read_data(df: pd.DataFrame, index: int, apply_filter = None, freq = None, freqmax = None, freqmin = None):
    """
    This function taks in a dataframe and some index from that dataframe. Can add filters if desired. 
    Returns stream object, p-wave arrival time, s-wave arrival time, and distance between event and seismogram in km
    """
    # read in metadata
    source_id = df.iloc[index]['event_id']
    net = df.iloc[index]['station_network_code']
    sta = df.iloc[index]['station_code']
    loc =  df.iloc[index]['station_location_code']
    cha = df.iloc[index]['trace_channel'] # switched form station channel code -> trace_channel
    sample_p = df.iloc[index]['trace_P_arrival_sample']
    sample_s = df.iloc[index]['trace_S_arrival_sample']
    trace_starttime = df.iloc[index]['trace_start_time']
    sample_rate = df.iloc[index]['trace_sampling_rate_hz']


    # calculate the distance


    # station latitude and longitude
    slat = df.iloc[index]['station_latitude_deg']
    slon = df.iloc[index]['station_longitude_deg']


    # source latitude and longitude
    lat0 = df.iloc[index]['source_latitude_deg'] 
    lon0 = df.iloc[index]['source_longitude_deg']


    # Use Obspy.gps2dist_azimuth to calc great circle distance in meters, azimuth A->B in degrees, azimuth B->A in degrees
    dist,az,baz=gps2dist_azimuth(lat0,lon0,slat,slon)



    # read the data

    # Store trace name in tn so that we can use it to grab waveform from hdf5 file
    tn = df.iloc[index]['trace_name']
    bucket, narray = tn.split('$')
    x, y, z = iter([int(i) for i in narray.split(',:')])
    # print(x,y,z)
    data = f['/data/%s' % bucket][x, :y, :z]


    # Create stream object

    stream = obspy.Stream()


    for i in range(3):
        tr = obspy.Trace()
        tr.data = data[i, :]
        tr.stats.sampling_rate = df.iloc[index]['trace_sampling_rate_hz']
        tr.stats.starttime = datetime.utcfromtimestamp(UTCDateTime(trace_starttime).timestamp)
        tr.stats.station = sta
        tr.stats.network = net
        tr.stats.location = loc
        if i == 0:
          tr.stats.channel = cha+"E"
        elif i == 1:
          tr.stats.channel = cha+"N"
        else :
          tr.stats.channel = cha+"Z"
        stream.append(tr)

    if apply_filter == 'bandpass':
        stream.filter(apply_filter, freqmin = freqmin, freqmax = freqmax)
    elif apply_filter is not None:
        stream.filter(apply_filter, freq = freq)

    #Phase pick and arrival time info

    starttime = datetime.utcfromtimestamp(UTCDateTime(trace_starttime).timestamp)
    x = [starttime + dt(seconds = i/sample_rate) for i in np.arange(15001)]
    p_time = sample_p/sample_rate 
    s_time = sample_s/sample_rate


    return stream, p_time, s_time, dist/1000

In [92]:
tr, tp, ts, dist = read_data(df, 3)
print(tr[0].stats.starttime)
tp1 = UTCDateTime(tr[0].stats.starttime) + tp
print(tp1)

2002-10-03T01:55:59.530000Z
2002-10-03T01:56:53.950000Z


In [109]:
# len(df['event_id'].unique())
ps_ratio = np.zeros(183909)
len(ps_ratio)


183909

In [110]:
# Calculate P/S ratio for 15943 explosions
ps_ratio = np.zeros(183909)

for i in range(200):
    # i = 0
    # read data, get stream object (tr), p-time (tp), s-time (ts), and dist btwn event and seismogram (dist)
    tr, tp, ts, dist = read_data(df, i)
    print(tr[0].stats.starttime)
    tp1 = UTCDateTime(tr[0].stats.starttime) + tp
    ts1 = UTCDateTime(tr[0].stats.starttime) + ts

    station = df.iloc[i]['station_code']
    if df.iloc[i]["station_location_code"] == "--":
        thissta = df.iloc[i]["station_network_code"] + "."+ df.iloc[i]["station_code"] + ".." + df.iloc[i]["trace_channel"] + "Z"
    else:
        thissta = df.iloc[i]["station_network_code"] + "."+ df.iloc[i]["station_code"] + "."+df.iloc[i]["station_location_code"] + "." + df.iloc[i]["trace_channel"] + "Z"


    inv0 = client.get_stations(network = df.iloc[i]["station_network_code"],\
                            station = df.iloc[i]["station_code"],\
                            location = "*",\
                            channel = df.iloc[i]["trace_channel"] + "*",\
                            starttime = UTCDateTime(2002,1,1),\
                            endtime = UTCDateTime(2022,12,1),\
                            level = "response")


    # remove instrumental response
    PAZ = inv0.get_response(thissta, tr[0].stats.starttime).get_paz()
    paz_sta = {'gain': PAZ.stage_gain,
                'poles': PAZ.poles,
                'sensitivity': PAZ.normalization_factor*PAZ.stage_gain,
                'zeros': PAZ.zeros}
    tr.simulate(paz_remove = paz_sta, water_level = 10)

    # filter 1 - 20 hz ??? make sure this is part of our project
    tr.filter('bandpass',freqmin = 1,freqmax = 20,corners = 2,zerophase = True)  # will do 1 - 20 **********
    # calculate envelope using the obspy function of the vertical component, and then its log
    # data_envelope = np.log10(obspy.signal.filter.envelope(tr[2].data))
    tr2 = tr[2].copy()
    # tr2.data[:] = data_envelope

    # Calculate the time where the envelope hits 2 times the noise floor. This is our NOISE VALUE to remove from P and S 

    noise = tr.copy()
    noise.trim(tp1 - 12, tp1 - 2)  
    noise_value = np.sum(noise[0].data**2 + noise[1].data**2 + noise[2].data**2)


    # Calculate the P-wave values

    wave_p = tr.copy()
    wave_p.trim(tp1 - 2, tp1 + 8)  
    wavep_value = np.sum(wave_p[0].data**2 + wave_p[1].data**2 + wave_p[2].data**2)

    # Calculate the S-wave values

    wave_s = tr.copy()
    wave_s.trim(ts1 - 2, ts1 + 8)  
    wave_s_value = np.sum(wave_s[0].data**2 + wave_s[1].data**2 + wave_s[2].data**2)

    # Put it all together- Calculate the P/S ratio

    ps_ratio[i] = np.sqrt(wavep_value - noise_value) / np.sqrt( wave_s_value - noise_value)
    print("Source type: ", df['source_type'][i], "P/S ratio:", ps_ratio[i])

# print(ps_ratio)

2002-10-03T01:55:59.530000Z
Source type:  earthquake P/S ratio: 0.0
2002-10-03T01:55:59.530000Z


c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


Source type:  earthquake P/S ratio: 0.0
2002-10-03T01:55:59.530000Z
Source type:  earthquake P/S ratio: 0.0
2002-10-03T01:55:59.530000Z


c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


Source type:  earthquake P/S ratio: 0.0
2002-09-26T06:59:14.860000Z
Source type:  earthquake P/S ratio: 0.0
2002-09-26T06:59:14.860000Z
Source type:  earthquake P/S ratio: 0.0
2002-09-26T06:59:14.860000Z
Source type:  earthquake P/S ratio: 0.0
2002-09-25T10:33:03.380000Z
Source type:  earthquake P/S ratio: 0.0
2002-09-25T10:33:03.380000Z
Source type:  earthquake P/S ratio: 0.0
2002-09-25T10:33:03.380000Z
Source type:  earthquake P/S ratio: 0.0
2002-09-25T10:33:03.380000Z


...

c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)
c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


Source type:  earthquake P/S ratio: 0.0
2002-05-27T19:32:48.770000Z
Source type:  earthquake P/S ratio: 0.0
2002-05-27T19:32:48.770000Z
Source type:  earthquake P/S ratio: 0.0
2002-05-27T19:32:48.770000Z
Source type:  earthquake P/S ratio: 0.0
2002-05-27T08:43:08.610000Z
Source type:  earthquake P/S ratio: 0.0
2002-05-27T08:43:08.610000Z


c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)
c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)
c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)
c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


Source type:  earthquake P/S ratio: 0.0
2002-05-27T08:43:08.610000Z
Source type:  earthquake P/S ratio: 0.0
2002-05-24T22:40:26.310000Z
Source type:  earthquake P/S ratio: 0.0
2002-05-20T10:43:48.960000Z
Source type:  earthquake P/S ratio: 0.0


c:\Users\wolfn\anaconda3\envs\PNSN\lib\site-packages\obspy\core\inventory\response.py:1935: UserWarning: More than one PolesZerosResponseStage encountered. Returning first one found.
  warnings.warn(msg)


In [ ]:
# 3 arrays : ps ratio, eventid, list of unique event id
# Take the median of all ps ratios for each uniqe event id
df.groupby('event_id')['ps_ratio'].median()
